<a href="https://colab.research.google.com/github/brendenwest/cis276/blob/main/7_time_series.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# Time Series Analysis

### Reading
- Murach's, Chapter 9
- https://wesmckinney.com/book/time-series.html

### Learning Outcomes

- Date & time conversion
- Basic Time Series data
- Handling date ranges & time periods
- Rolling time windows




### What is Time Series data

Time series data is an important form of structured data, and can be any data recorded repeatedly at many points in time.

Time series can have `fixed frequency` - data points occurring at regular intervals (e.g. hourly, daily, etc).

Time series can also be `irregular` without a fixed unit of time or offset between units.

Data points typically have a `timestamp` specifying an instant in time. Timestamp can be a `date`, a `time`, or a combination of both.


### Date & Time types

Python has built-in data types & modules for date and time data, such as `datetime`, `time`, & `calendar`.



In [ ]:
from datetime import datetime
now = datetime.now()
now
datetime.datetime(2023, 4, 12, 13, 9, 16, 484533)
now.year, now.month, now.day


### Pandas Time Series Basics

### Date Ranges & Frequencies

### Time Periods

### Moving Windows